<a href="https://colab.research.google.com/github/Satish316/KIET-CSC-DS-T3/blob/main/calories_burnt_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Sample dataset (replace with actual data)
data = pd.read_csv("/content/exercise_dataset.csv")

data = data.iloc[:, :-1]
data
data = data.rename(columns={'Activity, Exercise or Sport (1 hour)': 'activity', '130 lb': 'calories_130lb',
                            '155 lb': 'calories_155lb',
                            '180 lb': 'calories_180lb',
                            '205 lb': 'calories_205lb'})
df = data
df

,activity,calories_130lb,calories_155lb,calories_180lb,calories_205lb
0,"Cycling, mountain bike, bmx",502,598,695,791
1,"Cycling, <10 mph, leisure bicycling",236,281,327,372
2,"Cycling, >20 mph, racing",944,1126,1308,1489
3,"Cycling, 10-11.9 mph, light",354,422,490,558
4,"Cycling, 12-13.9 mph, moderate",472,563,654,745
...,...,...,...,...,...
243,General cleaning,207,246,286,326
244,"Cleaning, dusting",148,176,204,233
245,Taking out trash,177,211,245,279
246,"Walking, pushing a wheelchair",236,281,327,372


In [3]:
from sklearn.preprocessing import MultiLabelBinarizer

# Convert dataset to long format
weights = [130, 155, 180, 205]
long_format_data = []

for _, row in df.iterrows():
    activity_list = row["activity"].split(", ")  # Split multiple activities into a list
    for weight in weights:
        calories = row[f"calories_{weight}lb"]
        long_format_data.append([activity_list, weight, weight * 0.453592, calories])

# Create DataFrame in long format
long_df = pd.DataFrame(long_format_data, columns=["activity", "weight_lb", "weight_kg", "calories"])

# Apply MultiLabelBinarizer
mlb = MultiLabelBinarizer()
activity_encoded = mlb.fit_transform(long_df["activity"])
activity_df = pd.DataFrame(activity_encoded, columns=mlb.classes_)  # Class names as column headers

# Merge encoded activity data with the dataset
X = pd.concat([activity_df, long_df[["weight_lb"]]], axis=1)
y = long_df["calories"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Linear Regression Model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict and Evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Check the transformed feature columns
print("\nFeature Columns:")
print(X.columns)

Mean Squared Error: 2915.9529003165353

Feature Columns:
Index(['10 mph (6 min mile)', '10-11.9 mph', '10.9 mph (5.5 min mile)',
       '12-13.9 mph', '14-15.9 mph', '16-19 mph', '5 mph (12 minute mile)',
       '5.2 mph (11.5 minute mile)', '6 mph (10 min mile)',
       '6.7 mph (9 min mile)',
       ...
       'vigorous', 'walk', 'walking', 'walking and carrying clubs',
       'walking and pulling clubs', 'water calisthenics', 'water skiing',
       'wheelchair', 'yachting', 'weight_lb'],
      dtype='object', length=314)


In [4]:
import numpy as np

# Function to take user input, encode it, and predict calories burned
def predict_calories(user_activity, user_weight):
    # Convert user input activity into a list
    user_activity_list = user_activity.split(", ")  # Example: "Cycling, mountain bike, bmx" → ["Cycling", "mountain bike", "bmx"]

    # Encode activities using the trained MultiLabelBinarizer
    user_encoded = mlb.transform([user_activity_list])  # Transform into the same format as training data
    user_activity_df = pd.DataFrame(user_encoded, columns=mlb.classes_)  # Convert to DataFrame

    # Add weight column
    user_df = pd.concat([user_activity_df, pd.DataFrame({"weight_lb": [user_weight]})], axis=1)

    # Ensure input matches model training features (handle missing columns)
    user_df = user_df.reindex(columns=X.columns, fill_value=0)  # Add missing columns if any

    # Predict calories burned
    predicted_calories = model.predict(user_df)

    return predicted_calories[0]

# Example user input
user_activity = input("enter the activity:")  # User-provided activities
user_weight = int(input("enter the weight(lbs)")) # User-provided weight

# Predict calories burned
predicted_calories = predict_calories(user_activity, user_weight)
print("Predicted Calories Burned:", predicted_calories)


enter the activity:running
enter the weight(lbs)130
Predicted Calories Burned: 315.06839085257536


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['running'] will be ignored
  warnings.warn(
